In [4]:
import json
import requests
import smtplib
from email.mime.text import MIMEText

# API URL for fetching top news headlines (US, English, general category)
url = 'https://gnews.io/api/v4/top-headlines?category=general&lang=en&apikey=5bf103bb9df0e5ed3b6518429e7d240c'

# Fetch data from the API and parse JSON
try:
    response = requests.get(url)
    response.raise_for_status()  # Raise error if request fails
    data = response.json()
except Exception as e:
    print(f"Failed to fetch news data: {e}")
    data = {"articles": []}

# Keywords to detect political content in news articles
political_party_keywords = [
    # Party names (U.S. and international examples)
    "Democratic Party", "Republican Party", "Libertarian Party", "Green Party",
    "Labour Party", "Conservative Party", "Christian Democrats", "Social Democrats",
    "BJP", "Tories", "SPD", "CDU",

    # Abbreviations and shorthand
    "GOP", "DNC", "RNC",

    # Affiliation-related terms
    "party member", "partisan", "affiliated with", "endorsed by",
    "party leadership", "nominee", "candidate", "running under",
    "party lines", "along party lines", "party platform", "party-backed",

    # Contextual political structures
    "bipartisan", "partisan divide", "majority party", "minority party",
    "coalition", "opposition party", "ruling party", "split vote",
    "straight-ticket voting", "caucus", "primary election", "convention",

    # Common political verbs/actions
    "endorses", "nominates", "campaigns", "challenges", "criticizes",
    "backs", "aligns with", "fundraises", "votes along party lines", "withdraws support"
]

# Extract relevant political news
news_desc = []
for article in data.get("articles", []):
    description = article.get("description", "")
    title = article.get("title", "")
    for keyword in political_party_keywords:
        if keyword.lower() in description.lower() or keyword.lower() in title.lower():
            news_desc.append(f"{title}: {description}")
            break  # Avoid duplicate matches for the same article

# Create message body
if len(news_desc) > 0:
    message_body = "News Summary:\n\n" + "\n".join(news_desc)
else:
    message_body = "No substantial political news so far."

# Carrier email gateways for SMS/MMS
CARRIERS = {
    "att": "@mms.att.net",
    "tmobile": "@mms.tmobile.net",
    "verizon": "@vtext.com",
    "sprint": "@messaging.sprintpcs.com"
}

# Your Gmail credentials (App password is required for Gmail)
EMAIL = "USE_YOUR_OWN_EMAIL"
PASSWORD = "USE_YOUR_OWN_APP_PASSWORD"  # Use an App Password, not your actual password

def send_message(phone_number, carrier, message):
    """
    Sends a text message with the latest political news summary via Gmail SMTP.
    """
    carrier = carrier.lower()
    if carrier not in CARRIERS:
        raise ValueError(f"Unsupported carrier: {carrier}")

    recipient = f"{phone_number}{CARRIERS[carrier]}"
    msg = MIMEText(message)
    msg['Subject'] = "News Update"  # Email subject line
    msg['From'] = EMAIL
    msg['To'] = recipient

    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()
            server.login(EMAIL, PASSWORD)
            server.sendmail(EMAIL, recipient, msg.as_string())
        return True
    except Exception as e:
        print(f"Failed to send message: {e}")
        return False

# --- User Input ---
try:
    phone_number = input("Phone Number (Only numbers, no spaces/dashes): ").strip()
    if not phone_number.isdigit():
        raise ValueError("Phone number must contain only digits.")

    carrier = input("Pick one of the four options: att, tmobile, verizon, sprint: ").strip().lower()

    if send_message(phone_number, carrier, message_body):
        print(f"Message sent successfully!\n\nMessage:\n{message_body}")
    else:
        print("Failed to send the message.")
except Exception as e:
    print(f"Error: {e}")


Phone Number (Only numbers, no spaces/dashes): 6402030837
Pick one of the four options: att, tmobile, verizon, sprint: verizon
Message sent successfully!

Message:
No substantial political news so far.
